In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
import wandb

# Load the dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
# Normalize images
train_images, test_images = train_images / 255.0, test_images / 255.0
# Define class names
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
#intialize wanddb
wandb.init(project="deep_learning",name='mnist_dataset')
# Select one image per class
samples_per_class = {}
for img, label in zip(train_images, train_labels):
    if label not in samples_per_class:
        samples_per_class[label] = img
    if len(samples_per_class) == len(class_names):
        break
wandb.log({"Fashion-MNIST Sample Images": [wandb.Image(img, caption=class_names[label]) for label, img in samples_per_class.items()]})

# Plot images
fig, axes = plt.subplots(2, 5, figsize=(10, 5))
fig.suptitle("Fashion-MNIST Sample Images", fontsize=14)

for ax, (label, image) in zip(axes.flat, samples_per_class.items()):
    ax.imshow(image, cmap='gray')
    ax.set_title(class_names[label])
    ax.axis('off')

plt.tight_layout()
plt.show()
wandb.finish()

In [ ]:
#flatten the images
train_images = train_images.reshape(train_images.shape[0], -1)
test_images = test_images.reshape(test_images.shape[0], -1)

In [ ]:
import numpy as np

# Helper Functions
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Subtract max for numerical stability
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return (z > 0).astype(float)


def cross_entropy_loss(y_true, y_pred):
    n_samples = y_true.shape[0]
    logp = -np.log(y_pred[range(n_samples), y_true])
    return np.sum(logp) / n_samples
# Neural Network Class
class NeuralNetwork:
    def __init__(self, layer_sizes):
        self.layer_sizes = layer_sizes
        self.weights = []
        self.biases = []
        self._initialize_parameters()

    def _initialize_parameters(self):
        """Initialize weights and biases for all layers."""
        for i in range(len(self.layer_sizes) - 1):
            fan_in = self.layer_sizes[i]
            fan_out = self.layer_sizes[i + 1]
            # He initialization for hidden layers, Xavier for output layer
            if i < len(self.layer_sizes) - 2:  # Hidden layers use sigmoid
                scale = np.sqrt(2 / fan_in)
            else:  # Output layer uses softmax
                scale = np.sqrt(1 / fan_in)
            # Weight matrix: (fan_in, fan_out)
            weight = np.random.randn(fan_in, fan_out) * scale
            # Bias vector: (fan_out,)
            bias = np.zeros(fan_out)
            self.weights.append(weight)
            self.biases.append(bias)

    def forward(self, x):
        a = x  # Activation starts as the input
        # Process hidden layers
        for l in range(len(self.weights) - 1):
            z = np.dot(a, self.weights[l]) + self.biases[l]
            a = sigmoid(z)
        # Process output layer
        z = np.dot(a, self.weights[-1]) + self.biases[-1]
        output = softmax(z)
        return output

# Example Usage
if __name__ == "__main__":
    # Create a network: 784 input -> 256 hidden -> 128 hidden -> 10 output
    nn = NeuralNetwork([784, 256, 128, 10])
    # Simulate 5 input samples (e.g., Fashion-MNIST images)
    x = np.random.randn(5, 784)
    # Get probabilities
    probs = nn.forward(x)
    # Print probabilities for each sample
    for i, prob in enumerate(probs):
        print(f"Sample {i+1} probabilities: {prob}")